In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import random
from collections import Counter, defaultdict

In [ ]:
def clean_text(text, verbose=False): # Remove line breaks and extra spaces
    cleaned_text = ' '.join(text.replace('\n', ' ').replace('\r', ' ').split())
    if verbose == True:
        print('---------------------------------------------------')
        print('Text cleaned: ', cleaned_text)
        print('---------------------------------------------------')
    return cleaned_text # ' '.join(text.replace('\n', ' ').replace('\r', ' ').split())

In [ ]:
def get_top_songs(chart_soup, week, year, verbose=False):
    # Look for the table with the values
    chart_table = chart_soup.find('table', attrs={'width': '600'})
    if not chart_table:
        chart_table = chart_soup.find('table', attrs={'width': '650'})

    # If the table is not found
    if chart_table is None:
        if verbose == True:
            print('---------------------------------------------------')
            print('Chart Table:\n\n', chart_soup)
            print('---------------------------------------------------')
        print(f"[ERROR] No chart table found for Week {week}, Year {year}")
        return [] # LOOK FOR A BETTER WAY!

    rows = chart_table.find_all('tr')
    chart_data = []

    for idx, row in enumerate(rows):
        cols = row.find_all('td')
        if verbose == True:
            print('---------------------------------------------------')
            print('Scrap Columns: ', cols)
            print('---------------------------------------------------')
        if len(cols) >= 3: # Some rows may be decorative or headers
            position = idx + 1
            # We use an unlikely separator
            song_info = cols[2].get_text(separator="~~~", strip=True)
            song_info = clean_text(song_info, verbose=verbose)

            parts = [p.strip() for p in song_info.split("~~~") if p.strip()]

            # Special case: the second fragment is “- Artist”
            if len(parts) > 1 and parts[1].startswith('- '):
                title_and_artist = parts[0] + ' ' + parts[1]
                label = "~~~".join(parts[2:]).strip() if len(parts) > 2 else ""
            else:
                title_and_artist = parts[0]
                label = "~~~".join(parts[1:]).strip() if len(parts) > 1 else ""

            # Now separate by the last “ - ”
            if " - " in title_and_artist:
                split_idx = title_and_artist.rfind(" - ")
                title = title_and_artist[:split_idx].strip()
                artist = title_and_artist[split_idx + 3:].strip()
            else:
                title = title_and_artist.strip()
                artist = ""
                print(f'[ARTIST WARN] No artist found at position {position}: "{title}" - (Week {week}, Year {year})\nSong info: {song_info}')


            if verbose == True:
                print('---------------------------------------------------')
                print('Scraped Song Info: ', song_info)
                print('Scraped Song Parts: ', parts)
                print('Scraped Title and Artist: ', title_and_artist)
                print('Scraped Title: ', title)
                print('Scraped Artist: ', artist)
                print('Scraped Label: ', label)
                print('---------------------------------------------------')

            chart_data.append([year, week, position, title, artist, label])
        else:
            print('--------------------------------------------------------')
            print(f'[DATA WARN] Columns length != 3 at row {idx}: (Week {week}, Year {year})\nSong info: {cols}')
            print('--------------------------------------------------------')

    return chart_data

In [ ]:
def scrap_hits(year, week, verbose=False):
    # Format week number
    if week < 10:
        week = f'0{week}'

    reframed_hits = pd.DataFrame()   # <-- Empty DataFrame if there is no data

    # Make URL
    url_chart = f'http://www.mediatraffic.de/week{week}-{year}.htm'
    # print(url_chart)
    response_chart = requests.get(url_chart)

    # Check URL response (the structure of the link changes)
    if response_chart.status_code != 200:
        # Make new URL
        url_chart = f'http://www.mediatraffic.de/tracks-week{week}-{year}.htm'
        response_chart = requests.get(url_chart)

    # Check URL response (the structure of the link changes)
    if response_chart.status_code != 200:
        # Make new URL
        url_chart = f'http://www.mediatraffic.de/singles-week{week}-{year}.htm'
        response_chart = requests.get(url_chart)

    # Double check URL response
    if response_chart.status_code != 200 and week != 53:
        print(f'[WEEK WARN] No chart found at week {week}, year {year} (status: {response_chart.status_code})')
        return reframed_hits


    # If the page exists and can be processed
    chart_soup = BeautifulSoup(response_chart.content, 'html.parser')

    # Get the hits for that week
    week_hits = get_top_songs(chart_soup, week, year, verbose=verbose)

    if verbose == True:
        print('---------------------------------------------------')
        print('URL: ', url_chart)
        if week_hits:
            print('Week Hits: ', week_hits)
        print('---------------------------------------------------')

    # If there is data, add the year and week to each row
    if week_hits:
        # for row in week_hits:
        #     row.extend([week, year])

        # Convert data to pandas Dataframe
        reframed_hits = pd.DataFrame(week_hits, columns=['Year', 'Week', 'Position', 'Title', 'Artist', 'Label'])
        # hits = pd.concat([hits, reframed_hits], ignore_index=True)

    return reframed_hits

In [ ]:
#years_to_study = [1955, 2024]
years_to_study = [2021, 2024]
weeks = 53
hits = pd.DataFrame()
verbose = False

# Iterate over the specified years
for year in range(years_to_study[0], years_to_study[1] + 1):
    # print(year)
    # Iterate over the weeks of the year
    for week in range(1, weeks + 1):

        # Scrap the hits
        scraped_hits = scrap_hits(year, week, verbose=verbose)

        # Avoid the for loop to break
        if scraped_hits is None or scraped_hits.empty:
            continue

        hits = pd.concat([hits, scraped_hits], ignore_index=True)

        # Wait to avoid overloading the server
        time.sleep(random.uniform(1.2, 2.0))

print('✅ Finished! ✅')

[ERROR] No chart table found for Week 53, Year 2021
[ARTIST WARN] No artist found at position 6: "Running Up That Hill" - (Week 23, Year 2022)
Song info: Running Up That Hill~~~(A Deal With God) - Kate Bush~~~Noble & Brite Ltd. / EMI - 174.000 -~~~Largest Points Increase
[ARTIST WARN] No artist found at position 2: "Running Up That Hill" - (Week 24, Year 2022)
Song info: Running Up That Hill~~~(A Deal With God) - Kate Bush~~~Noble & Brite Ltd. / EMI - 306.000 -~~~Largest Points Increase
[ARTIST WARN] No artist found at position 2: "Running Up That Hill" - (Week 25, Year 2022)
Song info: Running Up That Hill~~~(A Deal With God) - Kate Bush~~~Noble & Brite Ltd. / EMI - 269.000
[ARTIST WARN] No artist found at position 3: "Running Up That Hill" - (Week 26, Year 2022)
Song info: Running Up That Hill~~~(A Deal With God) - Kate Bush~~~Noble & Brite Ltd. / EMI - 243.000
[ARTIST WARN] No artist found at position 3: "Running Up That Hill" - (Week 27, Year 2022)
Song info: Running Up That Hill~~

In [ ]:
hits.sample(5)

,Year,Week,Position,Title,Artist,Label
6338,2024,03,19,Luna,Feid feat. ATL Jacob,Universal Music Latino - 107.000
1577,2021,40,18,Woman,Doja Cat,Kemosabe - 142.000
5371,2023,31,12,Calm Down,Rema,4~~~Jonzing World / Mavin - 150.000
2306,2022,06,27,Srivalli,Sid Sriram & Devi Sri Prasad,Aditya Music India - 91.000 -~~~Largest Points...
7268,2024,26,29,Greedy,Tate McRae,4~~~RCA - 104.000


In [ ]:
hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8320 entries, 0 to 8319
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Year      8320 non-null   int64 
 1   Week      8320 non-null   object
 2   Position  8320 non-null   int64 
 3   Title     8320 non-null   object
 4   Artist    8320 non-null   object
 5   Label     8320 non-null   object
dtypes: int64(2), object(4)
memory usage: 390.1+ KB


## Count errors

### Insert Errors

In [ ]:
# Not updated

log_text = """
    [ERROR] No chart table found for Week 53, Year 1955
[ARTIST WARN] No artist found at position 9: "Band" - (Week 03, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 6: "Band" - (Week 04, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 6: "Band" - (Week 05, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 5: "Band" - (Week 06, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 5: "Band" - (Week 07, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 6: "Band" - (Week 08, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 4: "Band" - (Week 09, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 5: "Band" - (Week 10, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 6: "Band" - (Week 11, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 10: "Moritat (" - (Week 11, Year 1956)
Song info: Moritat (~~~Theme From 'The Three Penny Opera') - Dick Hyman~~~MGM
[ARTIST WARN] No artist found at position 7: "Moritat (" - (Week 12, Year 1956)
Song info: Moritat (~~~Theme From 'The Three Penny Opera') - Dick Hyman~~~MGM
[ARTIST WARN] No artist found at position 10: "Band" - (Week 12, Year 1956)
Song info: Band~~~Of Gold - Don Cherry~~~Columbia
[ARTIST WARN] No artist found at position 6: "Moritat (" - (Week 13, Year 1956)
Song info: Moritat (~~~Theme From 'The Three Penny Opera') - Dick Hyman~~~MGM
[ARTIST WARN] No artist found at position 6: "Moritat (" - (Week 14, Year 1956)
Song info: Moritat (~~~Theme From 'The Three Penny Opera') - Dick Hyman~~~MGM
[ARTIST WARN] No artist found at position 7: "Moritat (" - (Week 15, Year 1956)
Song info: Moritat (~~~Theme From 'The Three Penny Opera') - Dick Hyman~~~MGM
[ERROR] No chart table found for Week 53, Year 1956
[ERROR] No chart table found for Week 53, Year 1957
[ERROR] No chart table found for Week 53, Year 1959
[ERROR] No chart table found for Week 53, Year 1960
[ERROR] No chart table found for Week 53, Year 1961
[ARTIST WARN] No artist found at position 10: "Do You" - (Week 42, Year 1962)
Song info: Do You~~~Love Me - Contours~~~Gordy
[ERROR] No chart table found for Week 53, Year 1962
[ERROR] No chart table found for Week 53, Year 1963
[ARTIST WARN] No artist found at position 9: "Yesterday" - (Week 52, Year 1965)
Song info: Yesterday~~~Man - Chris Andrews~~~Vogue / Atco / Decca
[ERROR] No chart table found for Week 53, Year 1965
[ARTIST WARN] No artist found at position 7: "Yesterday" - (Week 01, Year 1966)
Song info: Yesterday~~~Man - Chris Andrews~~~Vogue / Atco / Decca
[ARTIST WARN] No artist found at position 5: "Yesterday" - (Week 02, Year 1966)
Song info: Yesterday~~~Man - Chris Andrews~~~Vogue / Atco / Decca
[ARTIST WARN] No artist found at position 6: "Yesterday" - (Week 03, Year 1966)
Song info: Yesterday~~~Man - Chris Andrews~~~Vogue / Atco / Decca
[ARTIST WARN] No artist found at position 4: "Yesterday" - (Week 04, Year 1966)
Song info: Yesterday~~~Man - Chris Andrews~~~Vogue / Atco / Decca
[ARTIST WARN] No artist found at position 5: "Yesterday" - (Week 05, Year 1966)
Song info: Yesterday~~~Man - Chris Andrews~~~Vogue / Atco / Decca
[ARTIST WARN] No artist found at position 4: "Yesterday" - (Week 06, Year 1966)
Song info: Yesterday~~~Man - Chris Andrews~~~Vogue / Atco / Decca
[ARTIST WARN] No artist found at position 8: "Yesterday" - (Week 07, Year 1966)
Song info: Yesterday~~~Man - Chris Andrews~~~Vogue / Atco / Decca
[ERROR] No chart table found for Week 53, Year 1966
[ERROR] No chart table found for Week 53, Year 1967
[ERROR] No chart table found for Week 53, Year 1969
[ERROR] No chart table found for Week 53, Year 1970
[ERROR] No chart table found for Week 53, Year 1971
[ERROR] No chart table found for Week 53, Year 1973
[ERROR] No chart table found for Week 53, Year 1974
[ERROR] No chart table found for Week 53, Year 1975
[ARTIST WARN] No artist found at position 8: "Lonely Night" - (Week 13, Year 1976)
Song info: Lonely Night~~~(Angel Face) - Captain & Tennille~~~A&M
[ARTIST WARN] No artist found at position 8: "Lonely Night" - (Week 14, Year 1976)
Song info: Lonely Night~~~(Angel Face) - Captain & Tennille~~~A&M
[ARTIST WARN] No artist found at position 8: "Lonely Night" - (Week 15, Year 1976)
Song info: Lonely Night~~~(Angel Face) - Captain & Tennille~~~A&M
[ARTIST WARN] No artist found at position 9: "Lonely Night" - (Week 16, Year 1976)
Song info: Lonely Night~~~(Angel Face) - Captain & Tennille~~~A&M
[ERROR] No chart table found for Week 53, Year 1976
[ARTIST WARN] No artist found at position 5: "Da Doo Ron Ron Shaun Cassidy" - (Week 29, Year 1977)
Song info: Da Doo Ron Ron Shaun Cassidy~~~Curb
[ARTIST WARN] No artist found at position 3: "Da Doo Ron Ron Shaun Cassidy" - (Week 30, Year 1977)
Song info: Da Doo Ron Ron Shaun Cassidy~~~Curb
[ARTIST WARN] No artist found at position 7: "Da Doo Ron Ron Shaun Cassidy" - (Week 31, Year 1977)
Song info: Da Doo Ron Ron Shaun Cassidy~~~Curb
[ARTIST WARN] No artist found at position 8: "Da Doo Ron Ron Shaun Cassidy" - (Week 32, Year 1977)
Song info: Da Doo Ron Ron Shaun Cassidy~~~Curb
[ARTIST WARN] No artist found at position 9: "That's Rock'n'Roll Shaun Cassidy" - (Week 44, Year 1977)
Song info: That's Rock'n'Roll Shaun Cassidy~~~Curb
[ARTIST WARN] No artist found at position 8: "That's Rock'n'Roll Shaun Cassidy" - (Week 45, Year 1977)
Song info: That's Rock'n'Roll Shaun Cassidy~~~Curb
[ERROR] No chart table found for Week 53, Year 1977
[ARTIST WARN] No artist found at position 1: "Mull Of Kintyre -" - (Week 05, Year 1978)
Song info: Mull Of Kintyre -~~~Paul McCartney &~~~Wings~~~Parlophone / Capitol - 1 week at No.1
[ARTIST WARN] No artist found at position 1: "Mull Of Kintyre -" - (Week 06, Year 1978)
Song info: Mull Of Kintyre -~~~Paul McCartney &~~~Wings~~~Parlophone / Capitol - 2 weeks at No.1
[ARTIST WARN] No artist found at position 1: "Mull Of Kintyre -" - (Week 07, Year 1978)
Song info: Mull Of Kintyre -~~~Paul McCartney &~~~Wings~~~Parlophone / Capitol - 3 weeks at No.1
[ARTIST WARN] No artist found at position 1: "Mull Of Kintyre -" - (Week 08, Year 1978)
Song info: Mull Of Kintyre -~~~Paul McCartney &~~~Wings~~~Parlophone / Capitol - 4 weeks at No.1
[ERROR] No chart table found for Week 53, Year 1979
[ERROR] No chart table found for Week 53, Year 1980
[ERROR] No chart table found for Week 53, Year 1981
[ERROR] No chart table found for Week 53, Year 1982
[ERROR] No chart table found for Week 53, Year 1984
[ERROR] No chart table found for Week 53, Year 1985
[ERROR] No chart table found for Week 53, Year 1986
[ERROR] No chart table found for Week 53, Year 1987
[ERROR] No chart table found for Week 53, Year 1989
[ERROR] No chart table found for Week 53, Year 1990
[ERROR] No chart table found for Week 53, Year 1991
[ERROR] No chart table found for Week 53, Year 1992
[ERROR] No chart table found for Week 53, Year 1994
[ERROR] No chart table found for Week 53, Year 1995
[ERROR] No chart table found for Week 53, Year 1996
[ERROR] No chart table found for Week 53, Year 1997
[ARTIST WARN] No artist found at position 9: "Sweetest Thing -" - (Week 02, Year 1999)
Song info: Sweetest Thing -~~~U2~~~Island
[ARTIST WARN] No artist found at position 7: "The Power Of Goodbye -" - (Week 04, Year 1999)
Song info: The Power Of Goodbye -~~~Madonna~~~Maverick
[ARTIST WARN] No artist found at position 7: "I'm Your Angel -" - (Week 05, Year 1999)
Song info: I'm Your Angel -~~~Celine Dion & R.Kelly~~~Epic
[ARTIST WARN] No artist found at position 1: "No Scrubs -" - (Week 18, Year 1999)
Song info: No Scrubs -~~~TLC~~~Laface / Arista - 1 week at No.1
[ARTIST WARN] No artist found at position 5: "Baby One More Time -" - (Week 24, Year 1999)
Song info: Baby One More Time -~~~Britney Spears~~~Jive
[ARTIST WARN] No artist found at position 11: "Baby One More Time -" - (Week 26, Year 1999)
Song info: Baby One More Time -~~~Britney Spears~~~Jive
[ARTIST WARN] No artist found at position 36: "Baby One More Time -" - (Week 31, Year 1999)
Song info: Baby One More Time -~~~Britney Spears~~~Jive
[ARTIST WARN] No artist found at position 11: "That Don't Impress Me Much -" - (Week 32, Year 1999)
Song info: That Don't Impress Me Much -~~~Shania Twain~~~Mercury
[ARTIST WARN] No artist found at position 3: "If You Had My Love -" - (Week 37, Year 1999)
Song info: If You Had My Love -~~~Jennifer Lopez~~~Work / Columbia
[ARTIST WARN] No artist found at position 2: "Bailamos" - (Week 38, Year 1999)
Song info: Bailamos~~~-~~~Enrique Iglesias~~~Interscope
[ARTIST WARN] No artist found at position 3: "Genie In A Bottle -" - (Week 38, Year 1999)
Song info: Genie In A Bottle -~~~Christina Aguilera~~~RCA -~~~Greatest Gainer, Airplay
[ARTIST WARN] No artist found at position 6: "If You Had My Love -" - (Week 38, Year 1999)
Song info: If You Had My Love -~~~Jennifer Lopez~~~Work / Columbia
[ARTIST WARN] No artist found at position 19: "Livin' La Vida Loca -" - (Week 38, Year 1999)
Song info: Livin' La Vida Loca -~~~Ricky Martin~~~C2
[ARTIST WARN] No artist found at position 2: "Blue -" - (Week 39, Year 1999)
Song info: Blue -~~~Eiffel 65~~~Skoopy / Bliss Co. -~~~Largest Points Increase - Greatest Gainer, Sales
[ARTIST WARN] No artist found at position 19: "Wild Wild West -" - (Week 39, Year 1999)
Song info: Wild Wild West -~~~Will Smith, Dru Hill & Kool Mo Dee~~~Columbia
[ARTIST WARN] No artist found at position 2: "Blue -" - (Week 40, Year 1999)
Song info: Blue -~~~Eiffel 65~~~Skoopy / Bliss Co.
[ARTIST WARN] No artist found at position 6: "Bailamos -" - (Week 40, Year 1999)
Song info: Bailamos -~~~Enrique Iglesias~~~Interscope
[ARTIST WARN] No artist found at position 3: "Blue -" - (Week 41, Year 1999)
Song info: Blue -~~~Eiffel 65~~~Skoopy / Bliss Co.
[ARTIST WARN] No artist found at position 5: "Bailamos -" - (Week 41, Year 1999)
Song info: Bailamos -~~~Enrique Iglesias~~~Interscope
[ARTIST WARN] No artist found at position 3: "Blue -" - (Week 42, Year 1999)
Song info: Blue -~~~Eiffel 65~~~Skoopy / Bliss Co.
[ARTIST WARN] No artist found at position 6: "Bailamos -" - (Week 42, Year 1999)
Song info: Bailamos -~~~Enrique Iglesias~~~Interscope
[ARTIST WARN] No artist found at position 3: "Blue -" - (Week 43, Year 1999)
Song info: Blue -~~~Eiffel 65~~~Skoopy / Bliss Co.
[ARTIST WARN] No artist found at position 7: "Bailamos -" - (Week 43, Year 1999)
Song info: Bailamos -~~~Enrique Iglesias~~~Interscope
[ARTIST WARN] No artist found at position 19: "If You Had My Love -" - (Week 43, Year 1999)
Song info: If You Had My Love -~~~Jennifer Lopez~~~Work / Columbia
[ARTIST WARN] No artist found at position 3: "Blue -" - (Week 44, Year 1999)
Song info: Blue -~~~Eiffel 65~~~Skoopy / Bliss Co.
[ARTIST WARN] No artist found at position 5: "Blue -" - (Week 45, Year 1999)
Song info: Blue -~~~Eiffel 65~~~Scoopy / Bliss Co.
[ARTIST WARN] No artist found at position 14: "Blue -" - (Week 50, Year 1999)
Song info: Blue -~~~Eiffel 65~~~Skoopy / Bliss Co.
[ARTIST WARN] No artist found at position 14: "Larger Than Life -" - (Week 51, Year 1999)
Song info: Larger Than Life -~~~Backstreet Boys~~~Jive
[ARTIST WARN] No artist found at position 15: "Mambo No.5 -" - (Week 51, Year 1999)
Song info: Mambo No.5 -~~~Lou Bega~~~RCA
[ARTIST WARN] No artist found at position 9: "Genie In A Bottle -" - (Week 52, Year 1999)
Song info: Genie In A Bottle -~~~Christina Aguilera~~~RCA
[ARTIST WARN] No artist found at position 16: "Larger Than Life -" - (Week 52, Year 1999)
Song info: Larger Than Life -~~~Backstreet Boys~~~Jive
[ERROR] No chart table found for Week 53, Year 1999
[ARTIST WARN] No artist found at position 17: "Thong Song -" - (Week 30, Year 2000)
Song info: Thong Song -~~~Sisqo~~~Dragon / Def Soul
[ARTIST WARN] No artist found at position 37: "Private Emotion" - (Week 31, Year 2000)
Song info: Private Emotion~~~-~~~Ricky Martin feat. Meja~~~Columbia
[ERROR] No chart table found for Week 53, Year 2000
[ERROR] No chart table found for Week 53, Year 2001
[ERROR] No chart table found for Week 53, Year 2002
[ERROR] No chart table found for Week 53, Year 2003
[ARTIST WARN] No artist found at position 33: "N" - (Week 33, Year 2005)
Song info: N~~~a Na Na Summer Girl - Porno Graffitti~~~Sony Music Entertainment Japan
[ARTIST WARN] No artist found at position 38: "Endless Story- Reira starring Yuna Ito" - (Week 38, Year 2005)
Song info: Endless Story- Reira starring Yuna Ito~~~Sony Music Japan
[ERROR] No chart table found for Week 53, Year 2005
[ERROR] No chart table found for Week 53, Year 2006
[ARTIST WARN] No artist found at position 40: "Headlines (Friendship Never Ends)- Spice Girls" - (Week 50, Year 2007)
Song info: Headlines (Friendship Never Ends)- Spice Girls~~~Virgin - 63.000
[ERROR] No chart table found for Week 53, Year 2007
[ERROR] No chart table found for Week 53, Year 2009
[ERROR] No chart table found for Week 53, Year 2010
[ERROR] No chart table found for Week 53, Year 2011
[ERROR] No chart table found for Week 53, Year 2012
[ERROR] No chart table found for Week 53, Year 2013
[ERROR] No chart table found for Week 53, Year 2014
[ARTIST WARN] No artist found at position 39: "Cheerleader" - (Week 05, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 74.000
[ARTIST WARN] No artist found at position 26: "Cheerleader" - (Week 06, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 95.000
[ARTIST WARN] No artist found at position 17: "Cheerleader" - (Week 07, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 111.000
[ARTIST WARN] No artist found at position 15: "Cheerleader" - (Week 08, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 123.000
[ARTIST WARN] No artist found at position 19: "Cheerleader" - (Week 09, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 116.000
[ARTIST WARN] No artist found at position 19: "Cheerleader" - (Week 10, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 112.000
[ARTIST WARN] No artist found at position 16: "Cheerleader" - (Week 11, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 124.000
[ARTIST WARN] No artist found at position 18: "Cheerleader" - (Week 12, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 126.000
[ARTIST WARN] No artist found at position 16: "Cheerleader" - (Week 13, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 128.000
[ARTIST WARN] No artist found at position 14: "Cheerleader" - (Week 14, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 135.000
[ARTIST WARN] No artist found at position 13: "Cheerleader" - (Week 15, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 135.000
[ARTIST WARN] No artist found at position 14: "Cheerleader" - (Week 16, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 151.000
[ARTIST WARN] No artist found at position 11: "Cheerleader" - (Week 17, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 142.000
[ARTIST WARN] No artist found at position 6: "Cheerleader" - (Week 18, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 231.000
[ARTIST WARN] No artist found at position 5: "Cheerleader" - (Week 19, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 232.000
[ARTIST WARN] No artist found at position 4: "Cheerleader" - (Week 20, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 240.000
[ARTIST WARN] No artist found at position 3: "Cheerleader" - (Week 21, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 239.000
[ARTIST WARN] No artist found at position 3: "Cheerleader" - (Week 22, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 238.000
[ARTIST WARN] No artist found at position 4: "Cheerleader" - (Week 23, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 238.000
[ARTIST WARN] No artist found at position 4: "Cheerleader" - (Week 24, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 254.000
[ARTIST WARN] No artist found at position 4: "Cheerleader" - (Week 25, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 270.000
[ARTIST WARN] No artist found at position 3: "Cheerleader" - (Week 26, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~Ultra Records - 281.000
[ARTIST WARN] No artist found at position 2: "Cheerleader" - (Week 27, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~2~~~Ultra Records - 297.000
[ARTIST WARN] No artist found at position 1: "Cheerleader" - (Week 28, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~2~~~Ultra Records - 310.000 - 1 week at No.1
[ARTIST WARN] No artist found at position 1: "Cheerleader" - (Week 29, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~2~~~Ultra Records - 420.000 - 2 weeks at No.1
[ARTIST WARN] No artist found at position 1: "Cheerleader" - (Week 30, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~2~~~Ultra Records - 326.000 - 3 weeks at No.1
[ARTIST WARN] No artist found at position 1: "Cheerleader" - (Week 31, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~2~~~Ultra Records - 311.000 - 4 weeks at No.1
[ARTIST WARN] No artist found at position 2: "Cheerleader" - (Week 32, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~2~~~Ultra Records - 301.000
[ARTIST WARN] No artist found at position 2: "Cheerleader" - (Week 33, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 280.000
[ARTIST WARN] No artist found at position 2: "Cheerleader" - (Week 34, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 272.000
[ARTIST WARN] No artist found at position 2: "Cheerleader" - (Week 35, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 257.000
[ARTIST WARN] No artist found at position 2: "Cheerleader" - (Week 36, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 231.000
[ARTIST WARN] No artist found at position 4: "Cheerleader" - (Week 37, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 211.000
[ARTIST WARN] No artist found at position 6: "Cheerleader" - (Week 38, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 182.000
[ARTIST WARN] No artist found at position 6: "Cheerleader" - (Week 39, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 167.000
[ARTIST WARN] No artist found at position 7: "Cheerleader" - (Week 40, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 153.000
[ARTIST WARN] No artist found at position 12: "Cheerleader" - (Week 41, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 132.000
[ARTIST WARN] No artist found at position 13: "Cheerleader" - (Week 42, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 121.000
[ARTIST WARN] No artist found at position 15: "Cheerleader" - (Week 43, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 111.000
[ARTIST WARN] No artist found at position 20: "Cheerleader" - (Week 44, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~3~~~Ultra Records - 99.000
[ARTIST WARN] No artist found at position 24: "Cheerleader" - (Week 45, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~4~~~Ultra Records - 88.000
[ARTIST WARN] No artist found at position 30: "Cheerleader" - (Week 46, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~4~~~Ultra Records - 78.000
[ARTIST WARN] No artist found at position 36: "Cheerleader" - (Week 47, Year 2015)
Song info: Cheerleader~~~(Felix Jaehn Remix) - Omi~~~4~~~Ultra Records - 72.000
[ARTIST WARN] No artist found at position 1: "High" - (Week 48, Year 2016)
Song info: High~~~Tension - AKB 48~~~King Records Japan - 564.000 - 1 week at No.1 - Hot Shot Debut
[ARTIST WARN] No artist found at position 13: "High" - (Week 49, Year 2016)
Song info: High~~~Tension - AKB 48~~~King Records Japan - 132.000
[ARTIST WARN] No artist found at position 33: "High" - (Week 50, Year 2016)
Song info: High~~~Tension - AKB 48~~~King Records Japan - 70.000
[ERROR] No chart table found for Week 53, Year 2016
[ARTIST WARN] No artist found at position 26: "Issue" - (Week 10, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 99.000
[ARTIST WARN] No artist found at position 27: "Issue" - (Week 11, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 100.000
[ARTIST WARN] No artist found at position 16: "Issue" - (Week 12, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 117.000
[ARTIST WARN] No artist found at position 17: "Issue" - (Week 13, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 120.000
[ARTIST WARN] No artist found at position 16: "Issue" - (Week 14, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 130.000
[ARTIST WARN] No artist found at position 14: "Issue" - (Week 15, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 134.000
[ARTIST WARN] No artist found at position 13: "Issue" - (Week 16, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 160.000
[ARTIST WARN] No artist found at position 37: "Xo Tour Llif3 -" - (Week 16, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 71.000
[ARTIST WARN] No artist found at position 12: "Issue" - (Week 17, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 165.000
[ARTIST WARN] No artist found at position 33: "Xo Tour Llif3 -" - (Week 17, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 88.000
[ARTIST WARN] No artist found at position 10: "Issue" - (Week 18, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 159.000
[ARTIST WARN] No artist found at position 31: "Xo Tour Llif3 -" - (Week 18, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 85.000
[ARTIST WARN] No artist found at position 11: "Issue" - (Week 19, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 148.000
[ARTIST WARN] No artist found at position 30: "Xo Tour Llif3 -" - (Week 19, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 88.000
[ARTIST WARN] No artist found at position 11: "Issue" - (Week 20, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 152.000
[ARTIST WARN] No artist found at position 31: "Xo Tour Llif3 -" - (Week 20, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 89.000
[ARTIST WARN] No artist found at position 11: "Issue" - (Week 21, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 153.000
[ARTIST WARN] No artist found at position 31: "Xo Tour Llif3 -" - (Week 21, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 89.000
[ARTIST WARN] No artist found at position 12: "Issue" - (Week 22, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 154.000
[ARTIST WARN] No artist found at position 29: "Xo Tour Llif3 -" - (Week 22, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 92.000
[ARTIST WARN] No artist found at position 14: "Issue" - (Week 23, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 159.000
[ARTIST WARN] No artist found at position 21: "Xo Tour Llif3 -" - (Week 23, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 103.000
[ARTIST WARN] No artist found at position 17: "Issue" - (Week 24, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 142.000
[ARTIST WARN] No artist found at position 22: "Xo Tour Llif3 -" - (Week 24, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 122.000
[ARTIST WARN] No artist found at position 18: "Issue" - (Week 25, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 131.000
[ARTIST WARN] No artist found at position 30: "Xo Tour Llif3 -" - (Week 25, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 91.000
[ARTIST WARN] No artist found at position 22: "Issue" - (Week 26, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 130.000
[ARTIST WARN] No artist found at position 31: "Xo Tour Llif3 -" - (Week 26, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 85.000
[ARTIST WARN] No artist found at position 20: "Issue" - (Week 27, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 124.000
[ARTIST WARN] No artist found at position 33: "Xo Tour Llif3 -" - (Week 27, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 83.000
[ARTIST WARN] No artist found at position 23: "Issue" - (Week 28, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 106.000
[ARTIST WARN] No artist found at position 33: "Xo Tour Llif3 -" - (Week 28, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 85.000
[ARTIST WARN] No artist found at position 27: "Issue" - (Week 29, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 98.000
[ARTIST WARN] No artist found at position 33: "Xo Tour Llif3 -" - (Week 29, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 88.000
[ARTIST WARN] No artist found at position 32: "Xo Tour Llif3 -" - (Week 30, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 82.000
[ARTIST WARN] No artist found at position 34: "Issue" - (Week 30, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 82.000
[ARTIST WARN] No artist found at position 35: "Xo Tour Llif3 -" - (Week 31, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 77.000
[ARTIST WARN] No artist found at position 36: "Issue" - (Week 31, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 76.000
[ARTIST WARN] No artist found at position 34: "Xo Tour Llif3 -" - (Week 32, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 73.000
[ARTIST WARN] No artist found at position 37: "Issue" - (Week 32, Year 2017)
Song info: Issue~~~s - Julia Michaels~~~Polydor / Republic - 71.000
[ARTIST WARN] No artist found at position 32: "Xo Tour Llif3 -" - (Week 33, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 75.000
[ARTIST WARN] No artist found at position 34: "Energetic -" - (Week 34, Year 2017)
Song info: Energetic -~~~Wanna One~~~CJ E&M Records South Korea - 75.000
[ARTIST WARN] No artist found at position 36: "Xo Tour Llif3 -" - (Week 34, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 72.000
[ARTIST WARN] No artist found at position 29: "Xo Tour Llif3 -" - (Week 36, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 86.000
[ARTIST WARN] No artist found at position 32: "Xo Tour Llif3 -" - (Week 37, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 78.000
[ARTIST WARN] No artist found at position 35: "Xo Tour Llif3 -" - (Week 38, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 71.000
[ARTIST WARN] No artist found at position 35: "Xo Tour Llif3 -" - (Week 39, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 69.000
[ARTIST WARN] No artist found at position 39: "Xo Tour Llif3 -" - (Week 40, Year 2017)
Song info: Xo Tour Llif3 -~~~Lil Uzi Vert~~~Generation Now / Atlantic - 62.000
[ERROR] No chart table found for Week 53, Year 2017
[ARTIST WARN] No artist found at position 13: "Stargazing- Travis Scott" - (Week 33, Year 2018)
Song info: Stargazing- Travis Scott~~~Cactus Jack / Grand Hustle / Epic - 135.000
[ARTIST WARN] No artist found at position 27: "Stargazing- Travis Scott" - (Week 34, Year 2018)
Song info: Stargazing- Travis Scott~~~Cactus Jack / Grand Hustle / Epic - 86.000
[ARTIST WARN] No artist found at position 35: "Stargazing- Travis Scott" - (Week 35, Year 2018)
Song info: Stargazing- Travis Scott~~~Cactus Jack / Grand Hustle / Epic - 69.000
[ERROR] No chart table found for Week 53, Year 2018
[ERROR] No chart table found for Week 53, Year 2019
[ARTIST WARN] No artist found at position 25: "Sta" - (Week 49, Year 2020)
Song info: Sta~~~y - BTS (Bangtan Boys)~~~Big Hit Entertainment - 112.000
"""

### Preprocessing

In [ ]:
# Processing
lines = log_text.strip().splitlines()
log_entries = []
current_entry = []

for line in lines:
    # If the line begins with an error or warning, it is a new message.
    if line.startswith('[ERROR]') or line.startswith('[ARTIST WARN]'):
        if current_entry:  # If there is a previous entry, save it.
            log_entries.append('\n'.join(current_entry))
            current_entry = []
        current_entry.append(line)
    else:
        # Otherwise, it is an additional line of the previous message (e.g., Song info).
        current_entry.append(line)

# Don't forget to add the latest entry.
if current_entry:
    log_entries.append('\n'.join(current_entry))

# Now log_entries is a list of strings, each one being a complete message.
print("First message example:\n", log_entries[0])
print("\nTotal messages:", len(log_entries))

Ejemplo de primer mensaje:
 [ERROR] No chart table found for Week 53, Year 1955

Total de mensajes: 225


### Counting errors

In [ ]:
# Counters
error_counter = Counter()
critical_years = defaultdict(set)
critical_weeks = defaultdict(list)
critical_positions = defaultdict(list)

for line in log_entries:
    if '[ERROR] No chart table found' in line:
        error_counter['No chart table'] += 1
        match = re.search(r'Week (\d+), Year (\d+)', line)
        if match:
            week, year = int(match.group(1)), int(match.group(2))
            critical_years[year].add(week)
            critical_weeks[year].append(week)
    elif '[ARTIST WARN] No artist found at position' in line:
        error_counter['No artist found'] += 1
        match = re.search(r'at position (\d+): ".*" - \(Week (\d+), Year (\d+)\)', line)
        if match:
            pos, week, year = int(match.group(1)), int(match.group(2)), int(match.group(3))
            critical_positions[(year, week)].append(pos)

print("Error counting:", error_counter)
print("Years with problematic weeks (table):")
for year, weeks in critical_years.items():
    print(f"{year}: weeks {sorted(weeks)}")
print("\nArtist's year/week errors:")
for (year, week), positions in critical_positions.items():
    print(f"Year {year}, Week {week}, Problematic Positions: {positions}")

Conteo de errores: Counter({'No artist found': 172, 'No chart table': 53})
Años con semanas problemáticas (tabla):
1955: semanas [53]
1956: semanas [53]
1957: semanas [53]
1959: semanas [53]
1960: semanas [53]
1961: semanas [53]
1962: semanas [53]
1963: semanas [53]
1965: semanas [53]
1966: semanas [53]
1967: semanas [53]
1969: semanas [53]
1970: semanas [53]
1971: semanas [53]
1973: semanas [53]
1974: semanas [53]
1975: semanas [53]
1976: semanas [53]
1977: semanas [53]
1979: semanas [53]
1980: semanas [53]
1981: semanas [53]
1982: semanas [53]
1984: semanas [53]
1985: semanas [53]
1986: semanas [53]
1987: semanas [53]
1989: semanas [53]
1990: semanas [53]
1991: semanas [53]
1992: semanas [53]
1994: semanas [53]
1995: semanas [53]
1996: semanas [53]
1997: semanas [53]
1999: semanas [53]
2000: semanas [53]
2001: semanas [53]
2002: semanas [53]
2003: semanas [53]
2005: semanas [53]
2006: semanas [53]
2007: semanas [53]
2009: semanas [53]
2010: semanas [53]
2011: semanas [53]
2012: seman

# Save the Dataset

In [ ]:
# hits.to_csv('hits.csv', index=False) # Make a temporal .csv File

## Save to PC

In [ ]:
# from google.colab import files

# files.download('hits.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Save to Google Drive

### Mount Google Drive

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Save in Google Drive

In [ ]:
# hits.to_csv('/content/drive/My Drive/hits.csv', index=False)

# # To save in a specific Folder
# # hits.to_csv('/content/drive/My Drive/mi_carpeta/hits.csv', index=False)